In [ ]:
# First of all, I will import the two libraries that will help me analyze the call data. 
# One of them is the poweful Pandas.🐼

import numpy as np
import pandas as pd

In [ ]:
# Secondly, I will import the libraries for data visualization.
# For a greater practicality, I will tap a code which will enable to watch the graphs after coding them.

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [ ]:
# Now I will read the file that I will analyze and visualize.
df = pd.read_csv('Call_Data.csv')

In [ ]:
# Let's extract some informations from this file.

df.info()   # We can see that there are more than 300,000 entries and 9 columns. 
            # There are numbers (floats and integers) and strings.

In [ ]:
# Let's check the head!

df.head()   # The data contains the following fields:

            # lat : String variable, Latitude
            # lng: String variable, Longitude
            # desc: String variable, Description of the Emergency Call
            # zip: String variable, Zipcode
            # title: String variable, Title
            # timeStamp: String variable, YYYY-MM-DD HH:MM:SS
            # twp: String variable, Township
            # addr: String variable, Address
            # e: String variable, Dummy variable (always 1)

In [ ]:
# Now I will answer some basic questions!

In [ ]:
# What're the top 5 zipcodes for  calls?

df.zip.value_counts().head(5)

In [ ]:
# What're the top 5 townships (twp) for calls?

df.twp.value_counts().head(5)

In [ ]:
# How many unique title codes are there?

df.title.nunique()

In [ ]:
# To better understand the informations whitin the data and posteriorly to use for a better prediction, ...
# ... is important create a new column from 'title' column with only the Reason.

df['Reason'] = df['title'].apply(lambda title: title.split(':')[0])

In [ ]:
# Lets check the result!

df.head(3)

In [ ]:
# What's the most common Reason for a call based in this new column?

df.Reason.value_counts()

In [ ]:
# Let's do the first graph visualization.
# " calls X Reason""

sns.countplot(x='Reason', data=df)

In [ ]:
# What's the data type of the objects in the 'timeStamp' column?

type(df['timeStamp'].iloc[0])

In [ ]:
# The timestamps are still strings. 
# I have to convert the column to DateTime objects with the purpose to better use in data analize.

df['timeStamp'] = pd.to_datetime(df['timeStamp'])

In [ ]:
# One more time, to good understand the informations and get a better prediction, ...
# ... is important create new columns, from 'timeStamp' column, called 'Hour', 'Month' and 'Day of Week'.

df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['Month'] = df['timeStamp'].apply(lambda time: time.month)
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)

# Let's see the result.

df.head(3) 

In [ ]:
# The 'Day of Week' column has only number instead of strings, such as 'Mon', 'Sun', 'Thu' and so on.
# I will solve this problem.

dmap = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
df['Day of Week'] = df['Day of Week'].map(dmap)

# Let's check if the problem was solved.

df.head(3) # Now we can see that the values in the 'Day of Week'changed from number to days of week.

In [ ]:
# Now I can use these new columns to do a new graph using 'Day of Week' column with the hue based in the Reason column.

sns.countplot(x='Day of Week',data=df,hue='Reason',palette='viridis')

# To relocate the legend

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# Now I will do the same thing for 'Month'

sns.countplot(x='Month',data=df,hue='Reason',palette='viridis')

# To relocate the legend

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) 

# We can notice that is missing some months: 9,10, and 11 are not there.

In [ ]:
# let's see if we can maybe fill in this information by plotting the information in another way.
# I will create a gropuby object called 'byMonth', where I group the DataFrame by the month column and use the count() method for aggregation.

byMonth = df.groupby('Month').count()

# Let's chck the result!

byMonth.head()

In [ ]:
# Now I will create a simple plot of the dataframe indicating the count of calls per month.
# Could be any column

byMonth['twp'].plot() #That's it. I got it!

In [ ]:
# I will see if I can use seaborn's lmplot() to create a linear fit on the number of calls per month. 
# I can't forget that I need to reset the index to a column.


sns.lmplot(x='Month',y='twp',data=byMonth.reset_index())

In [ ]:
# I will create a new column called 'Date' that contains the date from the timeStamp column.
# As I sad before, these news columns helps better unsderstand the informations.

df['Date']=df['timeStamp'].apply(lambda t: t.date())

# Let's check!

df.head(3)

In [ ]:
# Now groupby this Date column with the count() aggregate and create a plot of counts of calls.

plt.figure(figsize=(14,6)) # To better distinguish the legends.
df.groupby('Date').count()['twp'].plot()
plt.tight_layout()

In [ ]:
# Now I will recreate this plot, but creating 3 separate plots with each plot representing a Reason for the call.

plt.figure(figsize=(14,6))
df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot()
plt.title('SPIT')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(14,6))
df[df['Reason']=='Fire'].groupby('Date').count()['twp'].plot()
plt.title('Spam')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(14,6))
df[df['Reason']=='EMS'].groupby('Date').count()['twp'].plot()
plt.title('Normal')
plt.tight_layout()

# we can conclude that the 'Fire' has the lowest rate of call and 'EMS' has the uppest.  

In [ ]:
# Now let's move on to creating heatmaps with seaborn and our data. 
# We'll first need to restructure the dataframe, so that the columns become the Hours and the Index becomes the Day of the Week.

dayHour = df.groupby(by=['Day of Week','Hour']).count()['Reason'].unstack()

# Let'see...
dayHour.head()

In [ ]:
# Now I will create a HeatMap using this new DataFrame.

plt.figure(figsize=(12,6))
sns.heatmap(dayHour,cmap='viridis')

# We can notice that the higher volume of call happens Wednesday and Friday between 15h and 17h.

In [ ]:
# Now I will repeat these same plots and operations for Month as the column.
# I'm trying to get more and better informations and visualizations from data.

dayMonth = df.groupby(by=['Day of Week','Month']).count()['Reason'].unstack()

# Let's check!

dayMonth.head()

In [ ]:
# Heatmap

plt.figure(figsize=(12,6))
sns.heatmap(dayMonth,cmap='viridis')

# We can notice that 'Friday' appears again, but, joining the informations from the last heatmap, the higher calls happens in March, on friday and between 15h/17h.